# Deflection Profile Parametric Study
This file presents a parametric study based on the Deflection-Profile implementation provided in the file "deflection_profile.py" of the bmcs_beam package.

In [1]:
%matplotlib widget

In [2]:
from bmcs_cross_section.mkappa import MKappa
from bmcs_cross_section.cs_design.cs_layout import Reinforcement
from bmcs_beam.beam_config.boundary_conditions import BoundaryConditions, BoundaryConfig
from bmcs_beam.bending.deflection_profile import DeflectionProfile
from bmcs_beam.beam_config.beam_design import BeamDesign
from bmcs_cross_section.cs_design.cs_shape import TShape
from os.path import join
import numpy as np
import pandas as pd
import sympy as sp
import matplotlib.pyplot as plt
import bmcs_utils.api as bu

In [3]:
def plot_mk_ver(ax1, mc, ver_data):
    mc.plot_mk(ax1)
    k = ver_data[0]
    m = ver_data[1]
    ax1.plot(k, m, c= 'red', label= 'Experiment')
    ax1.legend()
#     plt.show()

In [4]:
def map_and_assign_params_for_rectangular_section(
        mc, # MKappa instance for which the params should be applied
        b, h, E, E_s, f_sy, eps_cr, rho, beta_tu, v_f, omega, psi, mu, alpha, lambda_cu):
    """This function map the params of the rectangular section from the type of the paper
    to the type of the model and assigns the values"""
    mc.cross_section_shape.B=b
    mc.cross_section_shape.H=h
    mc.E_cc=E
    mc.E_ct=E 
    mc.mu=mu
    mc.eps_cr=eps_cr
    mc.eps_cy=omega * eps_cr
    mc.eps_cu=lambda_cu * eps_cr
    mc.eps_tu=beta_tu * eps_cr
    mc.cross_section_layout.reinforcement = [Reinforcement(
        eps_sy=f_sy / E_s, # or eps_sy_j=psi * eps_cr,
        E=E_s,
        z=h * (1 - alpha),
        A=rho * b * h]
    

#### 1. Verification with Yang et al. (2010), http://dx.doi.org/10.1016/j.engstruct.2010.07.017, Steel reinforcement
![yang_2010.jpg](attachment:yang_2010.jpg)

In [39]:
# Curvature values in the Jao_Mobasher_2020 are 10*the source of the experiment (Yang_2010) 
excel_file = join('./verification_data/yang2010', 'fig_18', 'yang2010.xlsx')
data = pd.read_excel(excel_file, engine='openpyxl')
k= data['beam_R12_2_Curvature']
m= data['beam_R12_2_Moment']
ver_data = [k, m]


# R12-1,2 (rho = 0.#### 1. Verification with Yang et al. (2010), http://dx.doi.org/10.1016/j.engstruct.2010.07.017, Steel reinforcement
![yang_2010.jpg](attachment:yang_2010.jpg)6%)
mc = MKappa(n_m=100, n_kappa = 100)
mc.kappa_slider = 0.00001
mc.low_kappa = 0
mc.high_kappa = 0.00006
map_and_assign_params_for_rectangular_section(mc,
    beta_tu = 55, # Not sure where this is from, all other values match Mobasher
        b = 180, # 180 is the original, Saeed used b = 190
        h = 270,
        E = 46418, # according to Yang paper, according to Mobasher 46700
        E_s = 200000,
        f_sy = 600,
        eps_cr = 0.000384,
        rho = 0.0052, # exact according to Yang paper 253.4/(180*270), according to Mobasher 0.006
        v_f = 0.02,
        omega = 10.7,
        psi = 7.8,
        mu = 0.60,
        alpha = 0.87,
        lambda_cu = 11.7)
fig, (ax1) = plt.subplots(1, 1, figsize=(10, 5))
plot_mk_ver(ax1, mc, ver_data)

'[yang_2010.jpg]' is not recognized as an internal or external command,
operable program or batch file.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
# mc.interact()
bu.InteractiveWindow([mc, mc.matrix]).interact()

No solution The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
No solution The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
No solution The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
No solution The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.


In [7]:
def get_parametric_study(parameter_ranges):
    
    np.set_printoptions(precision=3)
    if np.ceil(len(parameter_ranges)/3).astype('int') == 1:
        nrows = np.ceil(len(parameter_ranges)/3).astype('int') + 1
    else:
        nrows = np.ceil(len(parameter_ranges)/3).astype('int')
    ncols= 3

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(7 * ncols, 5 * nrows))
    col = 0
    row = 0

       
    for param_name, param_range in parameter_ranges.items():

        mc = MKappa(n_m=100, n_kappa = 100)
        mc.low_kappa = 0
        mc.high_kappa = 0.00005
        map_and_assign_params_for_rectangular_section(mc,
            beta_tu = 21, # Not sure where this is from, all other values match Mobasher
                b = 180, # 180 is the original, Saeed used b = 190
                h = 270,
                E = 46418, # according to Yang paper, according to Mobasher 46700
                E_s = 200000,
                f_sy = 600,
                eps_cr = 0.000384,
                rho = 0.0052, # exact according to Yang paper 253.4/(180*270), according to Mobasher 0.006
                v_f = 0.02,
                omega = 10.7,
                psi = 7.8,
                mu = 0.55,
                alpha = 0.87,
                lambda_cu = 11.7)
        
        if param_name == 'B':
            for value in np.linspace(*param_range):
                setattr(mc.cross_section_shape, param_name, value)
                plot_mk_ver(axes[row, col], mc, ver_data)
        else:
            for value in np.linspace(*param_range):
                setattr(mc, param_name, value)
                plot_mk_ver(axes[row, col], mc, ver_data)
                
        if col < ncols - 1  :
            col +=1
        else:
            row += 1
            col = 0


In [8]:
# This dictionary provides the range and number of parameters
n = 3
parameter_ranges = {
        'B' : (150,180,n),
        'H': (250,290,n),
        'E_ct': (46418,60000,n),
        'E_cc': (46418,60000,n),
        'eps_cr': (0.000200,0.000384,n),
        'eps_cy': (-0.0041,-0.008,n),
        'eps_cu': (-0.0045,-0.008,n,int),
        'eps_tu': (0.008,0.01,3),
        'mu': (0.2,0.8,n),
        'z_j': ([0.],[35.],n),
        'A_j': ([253.4],[500.4],n),
        'E_j': ([200000.],[280000.], n),
        'eps_sy_j' : ([0.003], [0.009], n),
#         'n_m' : (2, 1000, 10)
    }

# ps = get_parametric_study(parameter_ranges)
# ps